# Ok


1. get jira to panda DF format
2. extract features
2.1 bag of words from description, creator, reporter
3. predict priopity
4. predict time estimate/ SP estimate

In [140]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import GaussianNB
from numpy import *
from sklearn.cross_validation import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.multiclass import OneVsOneClassifier
from sklearn.svm import LinearSVC
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import LabelEncoder

df = pd.read_csv('apache.csv', index_col=0, skiprows=0)
df

,projectCategory,created,priority,summary,description,creator,reporter
issue key,,,,,,,
ZOOKEEPER-2769,ZooKeeper,2017-04-28T19:44:50.000+0000,Major,Compiled documentation should not be under sou...,We have xml files that compile into our docume...,abrahamfine,abrahamfine
ZOOKEEPER-2768,ZooKeeper,2017-04-28T09:57:55.000+0000,Critical,Some ideas about four word command,Some ideas about four word command1) about con...,wangzhen,wangzhen
ZOOKEEPER-2767,ZooKeeper,2017-04-27T12:14:48.000+0000,Trivial,Correct the exception messages in X509Util if ...,In org.apache.zookeeper.common.X509Util.org.ap...,a72877,a72877
ZOOKEEPER-2766,ZooKeeper,2017-04-25T10:18:42.000+0000,Major,Quorum fails with java.io.EOFException,When I start a ZooKeeper ensemble comprising 3...,pkleindienst,pkleindienst
ZOOKEEPER-2765,ZooKeeper,2017-04-19T15:43:08.000+0000,Major,modern C++ client,"We should add a modern C++ (i.e. C++14, C++17,...",ecarter,ecarter
ZOOKEEPER-2764,ZooKeeper,2017-04-19T09:54:16.000+0000,Minor,"By default, only srvr four-letter word is on t...",Using the same Vagrant provisioning script as ...,ABchmn,ABchmn
ZOOKEEPER-2763,ZooKeeper,2017-04-19T06:34:02.000+0000,Minor,Utils.toCsvBuffer() omits leading 0 for bytes ...,"org.apache.jute.Utils.toCsvBuffer(), which con...",bberg,bberg
ZOOKEEPER-2762,ZooKeeper,2017-04-18T21:57:58.000+0000,Major,Multithreaded correctness Warnings,NaN,abrahamfine,abrahamfine
ZOOKEEPER-2761,ZooKeeper,2017-04-18T11:31:19.000+0000,Major,Build is packaged as uncompressed tar archive ...,This breaks my build scripts. Did work fine wi...,ABchmn,ABchmn


In [141]:

vect = TfidfVectorizer()
X = vect.fit_transform(df['summary'].values.astype('U')) 
yr = df['priority'].values.astype('U')


encoder = LabelEncoder()
encoder.fit(yr)
y = encoder.transform(yr)


In [142]:
TEST_SIZE = 0.2
RANDOM_STATE = 0
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE)


In [143]:
y_pred = OneVsOneClassifier(LinearSVC(random_state=0)).fit(X_train, y_train).predict(X_val)
confusion_matrix(y_val, y_pred)


array([[  1,   0,  18,   1,   0],
       [  0,   0,  10,   1,   0],
       [  1,   1, 110,  12,   0],
       [  0,   0,  30,   7,   0],
       [  0,   0,   6,   1,   1]])

In [108]:
y[:5]

array([2, 1, 4, 2, 2])

In [112]:
yr[:5]

array(['Major', 'Critical', 'Trivial', 'Major', 'Major'], 
      dtype='<U8')

In [113]:
encoder.classes_

array(['Blocker', 'Critical', 'Major', 'Minor', 'Trivial'], 
      dtype='<U8')

In [114]:
X_val

<200x4463 sparse matrix of type '<class 'numpy.float64'>'
	with 4783 stored elements in Compressed Sparse Row format>

In [6]:
def extract(review):
    '''extracts features from review'''
    result = {}
    for word in str(review).split():
        if word in result:
            result[word] += 1
        else:
            result[word] = 1
    return result

In [7]:
reviews = df['description']

In [8]:
X = []
y = []
words = []
# build an index of all occurring words
for review in reviews:
    for word in extract(review):
        if word not in words:
            words.append(word)


In [44]:
words

['We',
 'have',
 'xml',
 'files',
 'that',
 'compile',
 'into',
 'our',
 'documentation',
 'in',
 'src/docs/',
 'and',
 'precompiled',
 'docs/We',
 'should',
 'remove',
 'them',
 'only',
 'uncompiled',
 'under',
 'source',
 'control',
 'Some',
 'ideas',
 'about',
 'four',
 'word',
 'command1)',
 'cons,can',
 'we',
 'add',
 'command',
 'dimension',
 'data,such',
 'likenow/10.204.2.39:63943[1](queued=0,recved=7,sent=7,sid=0x154c32e8c2a5b8c,lop=PING,est=1483669807748,to=10000,l',
 'In',
 'org.apache.zookeeper.common.X509Util.org.apache.zookeeper.common.X509Util.createSSLContext',
 'exception',
 'messages',
 'contains',
 'keystore',
 'related',
 'instead',
 'of',
 'truststore',
 'for',
 'locati',
 'When',
 'I',
 'start',
 'a',
 'ZooKeeper',
 'ensemble',
 'comprising',
 '3',
 'nodes,',
 "I'm",
 'currently',
 'facing',
 'the',
 'following',
 'behavior:Two',
 'nodes',
 "(let's",
 'say',
 'node',
 '2',
 '3)',
 'out',
 'three',
 'their',
 'own',
 'quorum,',
 'finally',
 'one',
 'is',
 'modern',